In [1]:
%%writefile example2.txt
first
second line
the third line
this the fourth line

Overwriting example2.txt


In [2]:
from pyspark import SparkContext

In [3]:
sc = SparkContext()

In [4]:
sc.textFile('example2.txt')

example2.txt MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:-2

In [5]:
text_rdd = sc.textFile('example2.txt')

In [6]:
word = text_rdd.map(lambda line: line.split())

In [7]:
word.collect()

[['first'],
 ['second', 'line'],
 ['the', 'third', 'line'],
 ['this', 'the', 'fourth', 'line']]

In [8]:
text_rdd.collect()

['first', 'second line', 'the third line', 'this the fourth line']

In [9]:
# flat map everyhting will be a single element
flatword = text_rdd.flatMap(lambda line: line.split())

In [10]:
flatword.collect()

['first',
 'second',
 'line',
 'the',
 'third',
 'line',
 'this',
 'the',
 'fourth',
 'line']

In [11]:
%%writefile services.txt
#EventId    Timestamp    Customer   State    ServiceID    Amount
201       10/13/2017      100       NY       131          100.00
204       10/18/2017      700       TX       129          450.00
202       10/15/2017      203       CA       121          200.00
206       10/19/2017      202       CA       131          500.00
203       10/17/2017      101       NY       173          750.00
205       10/19/2017      202       TX       121          200.00

Overwriting services.txt


In [12]:
services = sc.textFile('services.txt')

In [13]:
services.take(2)

['#EventId    Timestamp    Customer   State    ServiceID    Amount',
 '201       10/13/2017      100       NY       131          100.00']

In [14]:
services.map(lambda line: line.split()).take(3)

[['#EventId', 'Timestamp', 'Customer', 'State', 'ServiceID', 'Amount'],
 ['201', '10/13/2017', '100', 'NY', '131', '100.00'],
 ['204', '10/18/2017', '700', 'TX', '129', '450.00']]

In [15]:
clean = services.map(lambda line: line[1:] if line[0] == '#' else line)

In [16]:
clean = clean.map(lambda line: line.split())

In [17]:
clean.collect()

[['EventId', 'Timestamp', 'Customer', 'State', 'ServiceID', 'Amount'],
 ['201', '10/13/2017', '100', 'NY', '131', '100.00'],
 ['204', '10/18/2017', '700', 'TX', '129', '450.00'],
 ['202', '10/15/2017', '203', 'CA', '121', '200.00'],
 ['206', '10/19/2017', '202', 'CA', '131', '500.00'],
 ['203', '10/17/2017', '101', 'NY', '173', '750.00'],
 ['205', '10/19/2017', '202', 'TX', '121', '200.00']]

In [24]:
# brag (State, Amount)
step1 = clean.map(lambda lst: (lst[3], lst[-1]))
# Reduce by key
step2 = step1.reduceByKey(lambda amt1,amt2 : float(amt1)+float(amt2))
# get rid of state amount title
step3 = step2.filter(lambda x: not x[0]=='State')
# Sort results by amount
step4 = step3.sortBy(lambda stAmount: stAmount[1], ascending=False)
# Action
step4.collect()

[('NY', 850.0), ('CA', 700.0), ('TX', 650.0)]

In [25]:
# tuple unpacking
x = ['ID', 'State', 'Amount']

In [26]:
def func1(lst):
    return lst[-1]
# not readable

In [27]:
def func2(Id_st_amt):
    #unpack values
    (Id,st,amt) = Id_st_amt
    return amt


In [28]:
func1(x)

'Amount'

In [30]:
func2(x)

'Amount'